In [1]:
import pandas as pd
import numpy as np

## 2.1 Разбиение набора данных

In [2]:
df=pd.read_csv('result_data.csv')

Разобъём набор данных таким образом, как это рекомендовано согласно документации `Sklearn`. А именно `30 на 70`. Как представленно в описании, такая выборка является оптимальной, поскольку абсолютное большинство данных должно находится при обучении модели, чтобы получить наиболее оптимизированную модель со стороны её точности

### Стратификация
При разделении стратифицируем данные, чтобы получить одинаковую в процентом соотношении выборку, чтобы не было перевеса на какой-то один класс и такая ситуация не повлияла на некорректное обучение модели

In [3]:
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,mean_new_cases,mean_new_deaths,Rt
0,MEX,North America,Mexico,2022-04-25,5733925.0,140.0,893.857,324134.0,5.0,27.143,...,1.38,75.05,0.779,0.0,0.0,0.0,0.0,6729.284877,371.968347,0.563587
1,MEX,North America,Mexico,2022-04-26,5735115.0,1190.0,835.000,324173.0,39.0,28.571,...,1.38,75.05,0.779,0.0,0.0,0.0,0.0,6729.284877,371.968347,0.563587
2,MEX,North America,Mexico,2022-04-27,5736579.0,1464.0,859.857,324221.0,48.0,31.000,...,1.38,75.05,0.779,0.0,0.0,0.0,0.0,6729.284877,371.968347,0.563587
3,MEX,North America,Mexico,2022-04-28,5737811.0,1232.0,882.286,324254.0,33.0,31.571,...,1.38,75.05,0.779,0.0,0.0,0.0,0.0,6729.284877,371.968347,0.563587
4,MEX,North America,Mexico,2022-04-29,5738769.0,958.0,1019.143,324294.0,40.0,37.286,...,1.38,75.05,0.779,0.0,0.0,0.0,0.0,6729.284877,371.968347,0.563587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183343,PCN,Oceania,Pitcairn,2021-06-16,0.0,0.0,0.000,0.0,0.0,0.000,...,0.00,0.00,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
183344,PCN,Oceania,Pitcairn,2021-06-17,0.0,0.0,0.000,0.0,0.0,0.000,...,0.00,0.00,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
183345,PCN,Oceania,Pitcairn,2021-06-18,0.0,0.0,0.000,0.0,0.0,0.000,...,0.00,0.00,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
183346,PCN,Oceania,Pitcairn,2021-06-19,0.0,0.0,0.000,0.0,0.0,0.000,...,0.00,0.00,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [4]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
df=df[df['Rt']<5].reset_index(drop=True)

### Определение переменной опасности

In [6]:
df1=df[df['Rt']<=0.7]
df1['Danger']=0

<ipython-input-6-868ce6df2db8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Danger']=0


In [7]:
df2=df[(df['Rt']>0.7) & (df['Rt']<=0.95)]
df2['Danger']=1

<ipython-input-7-c6c245977333>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Danger']=1


In [8]:
df3=df[df['Rt']>0.95]
df3['Danger']=2

<ipython-input-8-aaf4d467b356>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Danger']=2


In [9]:
df=pd.concat([df1, df2, df3]).reset_index(drop=True)

In [10]:
X=df[['new_cases', 'new_deaths', 'Rt']]
y=df['Danger']

In [11]:
#Получение выборок
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

## 2.3 Классифиткация 

Рассмотрим три модели классификации
### KNeighborsClassifier
Классификация на основе соседей - это тип обучения на основе экземпляров или необобщающего обучения: он не пытается построить общую внутреннюю модель, а просто сохраняет экземпляры обучающих данных. Классификация вычисляется простым большинством голосов ближайших соседей каждой точки: точке запроса назначается класс данных, который имеет наибольшее количество представителей среди ближайших соседей точки.

### RandomForestClassifier
Случайный лес — это метаоценка, которая соответствует ряду классификаторов дерева решений для различных подвыборок набора данных и использует усреднение для повышения точности прогнозирования и контроля переобучения. Размер подвыборки управляется параметром max_samples, если bootstrap=True (по умолчанию), в противном случае для построения каждого дерева используется весь набор данных
### GaussianNB
Наи́вный ба́йесовский классифика́тор — простой вероятностный классификатор, основанный на применении теоремы Байеса со строгими (наивными) предположениями о независимости. В зависимости от точной природы вероятностной модели, наивные байесовские классификаторы могут обучаться очень эффективно

## Матрикики
Рассмотрим две метрикики для оценивания модели классификации

### accuracy f1-score
Это гармоническое среднее значений точности и полноты. Возьмём её, потому что она дает лучшую оценку неправильно классифицированных случаев

### macro avg f1-score

macro avg f1-score пожалуй, самый простой из многочисленных методов усреднения. Макроусредненная оценка F1 (или макрооценка F1) вычисляется путем взятия среднего арифметического (также известного как невзвешенное среднее) всех оценок F1 для каждого класса. Этот метод будет взят, поскольку он обрабатывает все классы одинаково, независимо от их значений поддержки

## 2.4 Обучение

In [12]:
#Импорт моделей
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [13]:
#Обучение
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
preds=neigh.predict(X_test)
print(classification_report(preds, y_test))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86     18771
           1       0.53      0.60      0.56     10717
           2       0.82      0.80      0.81     28028

    accuracy                           0.78     57516
   macro avg       0.74      0.75      0.74     57516
weighted avg       0.78      0.78      0.78     57516



In [14]:
#Обучение
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_preds=rfc.predict(X_test)
print(classification_report(rfc_preds, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18337
           1       1.00      1.00      1.00     12075
           2       1.00      1.00      1.00     27104

    accuracy                           1.00     57516
   macro avg       1.00      1.00      1.00     57516
weighted avg       1.00      1.00      1.00     57516



In [15]:
#Обучение
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_preds=gnb.predict(X_test)
print(classification_report(gnb_preds, y_test))

              precision    recall  f1-score   support

           0       0.99      0.34      0.51     53237
           1       0.03      0.37      0.06      1111
           2       0.07      0.61      0.13      3168

    accuracy                           0.36     57516
   macro avg       0.36      0.44      0.23     57516
weighted avg       0.92      0.36      0.48     57516



### Вывод
Наиболее оптимальной моделью будет `KNeighborsClassifier` c accuracy f1-score = `0.78` и macro avg f1-score = `0.74`, поскольку по сравнению с другими он показал наилучший результат. `RandomForestClassifier` не будет взят, поскольку у него явное переобучение

## Отчёт
* 2.1 Разбиение набора данных - набор данныхз разбит на обучаюшую и тестовую выборки
* 2.3 Классификация - выбраны 3 алгоритма классификации
* 2.4 Обучение - произведена классификация по уровню опасности

In [ ]:
# Сохранение данных
df.to_csv('result_data.csv', encoding='utf-8-sig', index=False)